# Model Demo - KEY

## Environment Setup 

First, ensure that all imports below are accessible. These are required to run the demo code for the models

In [2]:
import os  # Python Native
from pathlib import Path  # Python Native

import keras  # pip install keras
import tensorflow as tf  # pip install tensorflow
from PIL import Image  # pip install pillow
import keras_hub  # pip install keras-hub
import huggingface_hub  # pip install huggingface_hub

2025-05-27 18:44:32.888760: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 18:44:33.017981: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-27 18:44:33.121419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748342673.205388    7792 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748342673.227404    7792 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748342673.415526    7792 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Then, to make the results readable, the list of classes is defined below and sorted.

The path to the test images is also defined below in `TEST_PATH`. Please ensure that it points to the correct path.

Note that the demo code does not filter out non-images within `TEST_PATH`, hence, you must ensure it only contains images for best results.

In [14]:
# The names of the classes
classes = [
    "Banded_Chlorosis",
    "Brown_Rust",
    "Brown_Spot",
    "Viral",
    "Yellow_Leaf",
    "Healthy",
]
# Sort the list to ensure they are in alphabetical order, the way the model knows it
classes.sort()

# Path to the folder that contains the test images
TEST_PATH = Path("./test/")

## Method 1: Convolutional Neural Network (CNN)

The trained model is stored on [Hugging Face](https://huggingface.co/ktrin-u/KEY-cnn/tree/main) at `ktrin-u/KEY-CNN`.

The code block below contains the code to run the CNN model on the test set. Comments detailing the each line are available.

Feel free to configure `IMAGE_COUNT`, `MODEL_PATH` and `OUTPUT_FILENAME` to your preferences.

In [ ]:
# Set IMAGE_COUNT to 0 or a negative number to import all images in the specified TEST_PATH
IMAGE_COUNT: int = 0
# Set the MODEL_PATH to refer to the hugging face repo where it was uploaded
MODEL_PATH = "hf://ktrin-u/KEY-cnn"
# Set the filename to which the results gets written to
OUTPUT_FILENAME = Path(f"./method1-results-{IMAGE_COUNT if IMAGE_COUNT > 0 else "ALL"}.csv")  # appends the image count to the end

# This loads the CNN model and prepares the output file to be written to
model = keras.models.load_model(MODEL_PATH)
output_file = open(OUTPUT_FILENAME, "w")  # Note that w will overwrite any file that matches with OUTPUT_FILENAME
output_file.write("image_filename,predicted_label\n") # Add header row

# Get the list of image names
image_names = os.listdir(TEST_PATH)
# Sort the image names as integers instead of strings to avoid 1, 11, 110 etc...
image_names.sort(key=lambda name: int(name.split(".")[0]))

# Go through each image found in TEST_PATH, open it, process it to be suitable for the model, then get the predicted class
# The result is a combination of the image name and its predicted class.
# Each result is written into the output file followed by a new line
for img_name in image_names[:IMAGE_COUNT if IMAGE_COUNT > 0 else None:]:
    # Use a try-except block to filter out non-images
    try:
        # Open the Image using PIL
        img = Image.open(TEST_PATH.joinpath(img_name))
    except Exception:
        print(f"{img_name} could not be opened by PIL.")
        continue  # go to next iteration
    # Convert the Image into a numpy array
    img_array = keras.utils.img_to_array(img)
    # Resize the image to the expected input of the model
    img_resized = keras.layers.Resizing(96, 96, pad_to_aspect_ratio=True)(img_array)
    # Add batch size axis expected by model
    img_resized = tf.expand_dims(img_resized, axis=0)
    # Call the model's predict function on the image
    prediction = model.predict(img_resized, batch_size=1, verbose="2")  # type: ignore ;
    # Get the index of the class with the highest value
    prediction_class_index = prediction.argmax()
    # print(prediction)
    # Format the result into the csv form;
    # class[prediction_class_indx] transforms the index into the class name, making it human readable
    result = f"{img_name},{classes[prediction_class_index]}"
    # Print the result
    print(result)
    # Write the result to the output file
    output_file.write(result + "\n")

# Once all the images have been given predictions, the output file is closed
output_file.close()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 50655.85it/s]


0.jpeg,Viral
1.jpeg,Viral
2.jpeg,Healthy
3.jpeg,Healthy
4.jpeg,Healthy
5.jpeg,Healthy
6.jpeg,Healthy
7.jpeg,Healthy
8.jpeg,Healthy
9.jpeg,Healthy
10.jpeg,Healthy
11.jpeg,Healthy
12.jpeg,Healthy
13.jpeg,Healthy
14.jpeg,Healthy
15.jpeg,Healthy
16.jpeg,Healthy
17.jpeg,Healthy
18.jpeg,Healthy
19.jpeg,Healthy
20.jpeg,Healthy
21.jpeg,Healthy
22.jpeg,Healthy
23.jpeg,Healthy
24.jpeg,Healthy
25.jpeg,Healthy
26.jpeg,Healthy
27.jpeg,Healthy
28.jpeg,Healthy
29.jpeg,Healthy
30.jpeg,Healthy
31.jpeg,Healthy
32.jpeg,Healthy
33.jpeg,Healthy
34.jpeg,Healthy
35.jpeg,Healthy
36.jpeg,Healthy
37.jpeg,Healthy
38.jpeg,Healthy
39.jpeg,Healthy
40.jpeg,Healthy
41.jpeg,Healthy
42.jpeg,Healthy
43.jpeg,Banded_Chlorosis
44.jpeg,Banded_Chlorosis
45.jpeg,Brown_Spot
46.jpeg,Banded_Chlorosis
47.jpeg,Banded_Chlorosis
48.jpeg,Banded_Chlorosis
49.jpeg,Banded_Chlorosis
50.jpeg,Banded_Chlorosis
51.jpeg,Banded_Chlorosis
52.jpeg,Banded_Chlorosis
53.jpeg,Banded_Chlorosis
54.jpeg,Banded_Chlorosis
55.jpeg,Banded_Chlorosis
56.jpeg

## Method 2: Vision Transformer (ViT)

The trained model is stored on [Hugging Face](https://huggingface.co) at [ktrin-u/KEY-ViT](https://huggingface.co/ktrin-u/KEY-ViT/tree/main).

The code block below contains the code to run the ViT model on the test set. 

It is nearly the exact same as the code block for **Method 1** above.

For convenience, each code black has been configured to be able to run independently of each other.

Similarly, feel free to configure `IMAGE_COUNT`, `MODEL_PATH` and `OUTPUT_FILENAME` to your preferences. 

In [ ]:
# Set IMAGE_COUNT to 0 or a negative number to import all images in the specified TEST_PATH
IMAGE_COUNT: int = -1
# Set the MODEL_PATH to refer to the hugging face repo where it was uploaded
MODEL_PATH = "hf://ktrin-u/KEY-ViT"
# Set the filename to which the results gets written to
OUTPUT_FILENAME = Path(f"./method2-results-{IMAGE_COUNT if IMAGE_COUNT > 0 else "ALL"}.csv")  # appends the image count to the end

# This loads the CNN model and prepares the output file to be written to
model = keras.models.load_model(MODEL_PATH)
output_file = open(OUTPUT_FILENAME, "w")  # Note that w will overwrite any file that matches with OUTPUT_FILENAME
output_file.write("image_filename,predicted_label\n") # Add header row

# Get the list of image names
image_names = os.listdir(TEST_PATH)
# Sort the image names as integers instead of strings to avoid 1, 11, 110 etc...
image_names.sort(key=lambda name: int(name.split(".")[0]))

# Go through each image found in TEST_PATH, open it, process it to be suitable for the model, then get the predicted class
# The result is a combination of the image name and its predicted class.
# Each result is written into the output file followed by a new line
for img_name in image_names[:IMAGE_COUNT if IMAGE_COUNT > 0 else None:]:
    # Use a try-except block to filter out non-images
    try:
        # Open the Image using PIL
        img = Image.open(TEST_PATH.joinpath(img_name))
    except Exception:
        print(f"{img_name} could not be opened by PIL.")
        continue  # go to next iteration
    # Convert the Image into a numpy array
    img_array = keras.utils.img_to_array(img)
    # Resize the image to the expected input of the model
    img_resized = keras.layers.Resizing(224, 224, pad_to_aspect_ratio=True)(img_array)
    # Add batch size axis expected by model
    img_resized = tf.expand_dims(img_resized, axis=0)
    # Call the model's predict function on the image
    prediction = model.predict(img_resized, batch_size=1, verbose="2")  # type: ignore ;
    # Get the index of the class with the highest value
    prediction_class_index = prediction.argmax()
    # print(prediction)
    # Format the result into the csv form;
    # class[prediction_class_indx] transforms the index into the class name, making it human readable
    result = f"{img_name},{classes[prediction_class_index]}"
    # Print the result
    print(result)
    # Write the result to the output file
    output_file.write(result + "\n")

# Once all the images have been given predictions, the output file is closed
output_file.close()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 49113.63it/s]


0.jpeg,Viral
1.jpeg,Viral
2.jpeg,Viral
3.jpeg,Healthy
4.jpeg,Healthy
5.jpeg,Healthy
6.jpeg,Healthy
7.jpeg,Healthy
8.jpeg,Healthy
9.jpeg,Healthy
10.jpeg,Healthy
11.jpeg,Healthy
12.jpeg,Healthy
13.jpeg,Healthy
14.jpeg,Healthy
15.jpeg,Healthy
16.jpeg,Healthy
17.jpeg,Healthy
18.jpeg,Healthy
19.jpeg,Healthy
20.jpeg,Healthy
21.jpeg,Healthy
22.jpeg,Healthy
23.jpeg,Healthy
24.jpeg,Healthy
25.jpeg,Healthy
26.jpeg,Healthy
27.jpeg,Healthy
28.jpeg,Healthy
29.jpeg,Healthy
30.jpeg,Healthy
31.jpeg,Healthy
32.jpeg,Healthy
33.jpeg,Healthy
34.jpeg,Healthy
35.jpeg,Healthy
36.jpeg,Healthy
37.jpeg,Healthy
38.jpeg,Healthy
39.jpeg,Healthy
40.jpeg,Healthy
41.jpeg,Healthy
42.jpeg,Healthy
43.jpeg,Banded_Chlorosis
44.jpeg,Yellow_Leaf
45.jpeg,Banded_Chlorosis
46.jpeg,Banded_Chlorosis
47.jpeg,Banded_Chlorosis
48.jpeg,Banded_Chlorosis
49.jpeg,Banded_Chlorosis
50.jpeg,Banded_Chlorosis
51.jpeg,Banded_Chlorosis
52.jpeg,Banded_Chlorosis
53.jpeg,Banded_Chlorosis
54.jpeg,Banded_Chlorosis
55.jpeg,Banded_Chlorosis
56.jpeg,